## IMPORTS 

In [24]:
%matplotlib inline
#
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist
import pandas as pd
from sklearn.metrics import confusion_matrix

In [25]:
# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [26]:
#Check TensorFlow Version
tf.__version__

'1.13.1'

In [27]:
#Check Keras Version
tf.keras.__version__

'2.2.4-tf'

## IMPORT TWEET TEXT DATA


In [28]:
#Import
dem_train_text1 = pd.read_csv('../data_master/nlp_feed_data/dem_tweets.csv')                        

rep_test_text1 = pd.read_csv('../data_master/nlp_feed_data/rep_tweets.csv')

#Drop and show df
dem_train_text2 = dem_train_text1.drop(['Unnamed: 0'], axis=1)
print('DEMOCRAT TRAIN TWEETS')
print(dem_train_text2.head())
print(type(dem_train_text2)) 
      
rep_test_text2 = rep_test_text1.drop(['Unnamed: 0'], axis=1)
print('REPUBLICAN TEST TWEETS')
print(rep_test_text2.head())
print(type(rep_test_text2))


DEMOCRAT TRAIN TWEETS
                                               tweet
0  We're in a battle for the soul of this nation....
1  Today marks 50 years since the Stonewall upris...
2  I’ve fought my heart out to ensure that civil ...
3  At last night’s first Democratic debate, I lai...
4  I just got off the debate stage, where I laid ...
<class 'pandas.core.frame.DataFrame'>
REPUBLICAN TEST TWEETS
                                               tweet
0  NO American should worry about the cost of lif...
1  ALL American families deserve to live in safe ...
2  A targeted attack on our democracy is no laugh...
3  House Democrats are FOCUSED on protecting our ...
4  Mental health care is health care. We must pro...
<class 'pandas.core.frame.DataFrame'>


## CONVERT DATAFRAMES TO STRING FOR PREP TO FEED TO NLP

In [29]:
#Dataframe to list then list to strings
rep_test_text = rep_test_text.values.tolist()
for i in range(len(rep_test_text)):
    rep_test_text[i] = str(rep_test_text[i])
dem_train_text = dem_train_text.values.tolist()
for i in range(len(dem_train_text)):
    dem_train_text[i] = str(dem_train_text[i])
    
#Show strings
print('REPUBLICAN TEST LIST LENGTH')
print(len([rep_test_text]))
print(type(rep_test_text))
print('DEMOCRAT TRAIN LIST LENGTH')
print(len([dem_train_text]))
print(type(dem_train_text))

NameError: name 'rep_test_text' is not defined

## (NOT NEEDED BUT WANT TO KEEP IN CASE OF PIVOT)

In [30]:
IMPORT CLASSIFIER DATA

#Import
#dem_train_y = pd.read_csv('../data_master/nlp_feed_data/dem_classifier.csv')                        

#rep_train_x = pd.read_csv('../data_master/nlp_feed_data/rep_classifier.csv')

#Drop and show
#dem_train_y = dem_train_y.drop(['Unnamed: 0'], axis=1)
#print('DEMOCRAT CLASSIFIER')
#print(dem_train_y.head())
#print(type(dem_train_y))
      
#rep_train_x = rep_train_x.drop(['Unnamed: 0'], axis=1)
#print('REPUBLICAN CLASSIFIER')
#print(rep_train_x.head())
#print(type(rep_train_x))

CONVERT DATAFRAMES TO LIST FOR PREP FEED TO NLP

#rep_train_x = rep_train_x.values.tolist()
#for i in range(len(rep_train_x)):
#    rep_train_x[i] = str(rep_train_x[i])
#dem_train_y = dem_train_y.values.tolist()

#print('REPUBLICAN LIST LENGTH')
#print(len([rep_train_x]))
#print(type(rep_train_x))
#print('DEMOCRAT LIST LENGTH')
#print(len([dem_train_y]))
#print(type(dem_train_y))

SyntaxError: invalid syntax (<ipython-input-30-feb4758469ec>, line 1)

## CREATE DATAFRAMES WITH CLASSIFIER VALUE 1 OR 0

In [31]:
dem_train_y = [i for i in np.arange(1,9055,1)]
print('Interger Return off of DF initial build (7th posistion)')
print(dem_train_y[7])
for n, i in enumerate(dem_train_y):
              dem_train_y[n] = 1
print('Interger Return off of DF refactored (0) build')
print(dem_train_y[7])
dem_train_y

#list to DF 
dem_train_y_df = pd.DataFrame(dem_train_y)


rep_test_x = [i for i in np.arange(1,14449,1)]
print('Interger Return off of DF initial build (7th posistion)')
print(rep_test_x[7])
for n, i in enumerate(rep_test_x):
              rep_test_x[n] = 0
print('Interger Return off of DF refactored (1) build')
print(type(rep_test_x))
#list to DF 
rep_test_x_df = pd.DataFrame(rep_test_x)
rep_test_text2
rep_test_x_df

Interger Return off of DF initial build (7th posistion)
8
Interger Return off of DF refactored (0) build
1
Interger Return off of DF initial build (7th posistion)
8
Interger Return off of DF refactored (1) build
<class 'list'>


,0
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


## DATA REWORK

In [32]:
#rep_tweet_0 = pd.concat([rep_test_text2, rep_test_x_df], axis=1, join='union')
#dem_tweet_1 = pd.concat([dem_train_text2, dem_train_y_df], axis=1, join='union')
#merged = pd.merge(rep_test_text2, rep_test_x_df, on=['tweet','0'])
rep_tc_0 = pd.merge(rep_test_text2, rep_test_x_df, left_index=True, right_index=True)
dem_tc_1 = pd.merge(dem_train_text2, dem_train_y_df, left_index=True, right_index=True)
dem_tc_1.head()


,tweet,0
0,We're in a battle for the soul of this nation....,1
1,Today marks 50 years since the Stonewall upris...,1
2,I’ve fought my heart out to ensure that civil ...,1
3,"At last night’s first Democratic debate, I lai...",1
4,"I just got off the debate stage, where I laid ...",1


In [48]:
dem_rep_all = dem_tc_1.append(rep_tc_0, ignore_index=True)
dem_rep_all.head()


,tweet,0
0,We're in a battle for the soul of this nation....,1
1,Today marks 50 years since the Stonewall upris...,1
2,I’ve fought my heart out to ensure that civil ...,1
3,"At last night’s first Democratic debate, I lai...",1
4,"I just got off the debate stage, where I laid ...",1


In [47]:
dem_rep_allv2 = dem_rep_all.drop("tweet", axis=1)
dem_rep_allv2.head()

,0
0,1
1,1
2,1
3,1
4,1


In [57]:
dem_rep_allv2 = dem_rep_allv2.rename(columns={0: 1})
dem_rep_allv2

,1
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


In [62]:
dem_rep_allv7 = dem_rep_allv2.rename(columns={1: 2})
dem_rep_allv7

,2
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


In [63]:
dem_rep_allv3 = pd.merge(dem_rep_all, dem_rep_allv2, left_index=True, right_index=True)
dem_rep_allv5 = pd.merge(dem_rep_allv3, dem_rep_allv7, left_index=True, right_index=True)
dem_rep_allv5

,tweet,0,1,2
0,We're in a battle for the soul of this nation....,1,1,1
1,Today marks 50 years since the Stonewall upris...,1,1,1
2,I’ve fought my heart out to ensure that civil ...,1,1,1
3,"At last night’s first Democratic debate, I lai...",1,1,1
4,"I just got off the debate stage, where I laid ...",1,1,1
5,That’s a wrap! Thanks for following along with...,1,1,1
6,"""There's not a single thing the American peopl...",1,1,1
7,We know NATO will fall apart if Donald Trump i...,1,1,1
8,Retweet if you're proud to be on #TeamJoe! #De...,1,1,1
9,The Biden administration will make the largest...,1,1,1


In [65]:
X = dem_rep_allv5.drop("tweet", axis=1)
y = dem_rep_allv5["tweet"]
print(X.shape, y.shape)

(23502, 3) (23502,)


In [66]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

## COMBINE DATA SETS FOR USE LATER

In [62]:
data_text = dem_train_text + rep_test_text

## TOKENIZER

In [63]:
num_words = 10000

In [64]:
tokenizer = Tokenizer(num_words=num_words)

In [65]:
%%time
tokenizer.fit_on_texts(data_text)

CPU times: user 870 ms, sys: 8.11 ms, total: 878 ms
Wall time: 877 ms


In [66]:
if num_words is None:
    num_words = len(tokenizer.word_index)

In [67]:
tokenizer.word_index

{'the': 1,
 'to': 2,
 'co': 3,
 't': 4,
 'https': 5,
 'of': 6,
 'a': 7,
 'and': 8,
 'in': 9,
 'for': 10,
 'is': 11,
 'amp': 12,
 'n': 13,
 "'": 14,
 'on': 15,
 'that': 16,
 'we': 17,
 'our': 18,
 'with': 19,
 'are': 20,
 'this': 21,
 'will': 22,
 'be': 23,
 'it': 24,
 'have': 25,
 'you': 26,
 'i': 27,
 'they': 28,
 'not': 29,
 'from': 30,
 'by': 31,
 'at': 32,
 'their': 33,
 'has': 34,
 'people': 35,
 'as': 36,
 'all': 37,
 'who': 38,
 'but': 39,
 'he': 40,
 'more': 41,
 'about': 42,
 'his': 43,
 'was': 44,
 'an': 45,
 'us': 46,
 'your': 47,
 'or': 48,
 'my': 49,
 's': 50,
 "'the": 51,
 'up': 52,
 'now': 53,
 '”': 54,
 'no': 55,
 'nhttps': 56,
 'do': 57,
 'u': 58,
 'out': 59,
 'them': 60,
 'must': 61,
 'today': 62,
 'country': 63,
 'president': 64,
 'can': 65,
 '–': 66,
 'time': 67,
 'just': 68,
 'american': 69,
 'should': 70,
 'one': 71,
 'americans': 72,
 'what': 73,
 'great': 74,
 'so': 75,
 'make': 76,
 'venezuela': 77,
 'if': 78,
 'get': 79,
 'trump': 80,
 'need': 81,
 'tax': 82,


In [68]:
#Use the tokenizer to convert all democrat tweets in the training-set to lists of these tokens.
dem_text_train_tokens = tokenizer.texts_to_sequences(dem_train_text)

In [69]:
np.array(dem_text_train_tokens[1])

array([ 244, 4000,  829,  105,  300,    1, 6221,  240,  110, 1117, 2517,
        681,    7,  165,   15, 4165, 1624,    8, 2191,   18,  166, 1749,
         17,   61,  222,   18,  122,   10,  350, 1429, 1240,  355,  288,
         11, 1262,   19,    1,  743,    8,  861,   28,  352,    5,    4,
          3])

In [70]:
#Also need to convert the tweets in the test-set to tokens.
rep_test_tokens = tokenizer.texts_to_sequences(rep_test_text)

## PADDING AND TRUNCATING DATA

In [71]:
#Count the number of tokens in all the sequences in the data-set.
num_tokens = [len(tokens) for tokens in dem_text_train_tokens + rep_test_tokens]
num_tokens = np.array(num_tokens)


In [72]:
#Average number of tokens in a sequence
np.mean(num_tokens)

30.565441239043487

In [73]:
#Maximum number of tokens in a sequence
np.max(num_tokens)

62

In [74]:
#Max number of tokens we will allow is set to the average plus 2 standard deviations
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

56

In [75]:
#Covers about 99% of the data-set
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.9950217002808271

## NVM ACTUAL PADDING HERE

In [76]:
pad = 'pre'

In [77]:
x_train_pad = pad_sequences(dem_text_train_tokens, maxlen=max_tokens,
                            padding=pad, truncating=pad)

In [78]:
x_test_pad = pad_sequences(rep_test_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

In [79]:
#Matrix for the train-set
x_train_pad.shape

(9054, 56)

In [80]:
#Matrix for the test-set
#
# !!! EXAMPLE HAS TEST AND TRAIN SHAPE THE EXACT SAME !!!
#
x_test_pad.shape

(14448, 56)

In [81]:
#Squence of tokens
np.array(rep_test_tokens[1])

array([1753,   69,   93,  352,    2,  150,    9,  407,  394,   13, 1815,
        108,   25,  325, 5584, 4967, 1023,    2,  480,   18, 3574,  557,
         10, 1454,    2,  236,   39,  240,   20,  742,    8,   90,  147,
          2,   57,   33,  184,   13,   56,    4,    3])

In [82]:
#Squence of tokens
np.array(dem_text_train_tokens[1])

array([ 244, 4000,  829,  105,  300,    1, 6221,  240,  110, 1117, 2517,
        681,    7,  165,   15, 4165, 1624,    8, 2191,   18,  166, 1749,
         17,   61,  222,   18,  122,   10,  350, 1429, 1240,  355,  288,
         11, 1262,   19,    1,  743,    8,  861,   28,  352,    5,    4,
          3])

In [83]:
#Padded to create the following sequence
#
# !!! LOOKS DIFFERENT FROM THE EXAMPLE !!!
#
rep_test_tokens[1]

[1753,
 69,
 93,
 352,
 2,
 150,
 9,
 407,
 394,
 13,
 1815,
 108,
 25,
 325,
 5584,
 4967,
 1023,
 2,
 480,
 18,
 3574,
 557,
 10,
 1454,
 2,
 236,
 39,
 240,
 20,
 742,
 8,
 90,
 147,
 2,
 57,
 33,
 184,
 13,
 56,
 4,
 3]

## TOKENIZER INVERSE MAP

In [84]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [85]:
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

In [86]:
tokens_to_string(dem_text_train_tokens[1])

"'today marks 50 years since the uprising where those brave individuals took a stand on christopher st and changed our world forever we must continue our fight for full lgbtq equality until everyone is treated with the dignity and respect they deserve https t co"

In [87]:
tokens_to_string(rep_test_tokens[1])

"'all american families deserve to live in safe communities n nhouse democrats have passed 2️⃣ gunsafety bills to ensure our children’s safety for generations to come but where are senatemajldr and senate republicans to do their job n nhttps t co"

## CREATE THE RECURRENT NEURAL NETWORK

In [88]:
model = Sequential()

In [89]:
embedding_size = 8

In [90]:
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))

In [91]:
model.add(GRU(units=16, return_sequences=True))

In [92]:
model.add(GRU(units=8, return_sequences=True))

In [93]:
model.add(GRU(units=4))

In [94]:
model.add(Dense(1, activation='sigmoid'))

In [95]:
optimizer = Adam(lr=1e-3)

In [96]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [97]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 56, 8)             80000     
_________________________________________________________________
gru_3 (GRU)                  (None, 56, 16)            1200      
_________________________________________________________________
gru_4 (GRU)                  (None, 56, 8)             600       
_________________________________________________________________
gru_5 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5         
Total params: 81,961
Trainable params: 81,961
Non-trainable params: 0
_________________________________________________________________


## TRAIN THE RECURRENT NERUAL NETWORK

In [98]:
%%time
model.fit(x_train_pad, dem_train_y,
          validation_split=0.05, epochs=3, batch_size=64)

Train on 8601 samples, validate on 453 samples
Epoch 1/3
8601/8601 [==============================] - 15s 2ms/sample - loss: 0.2525 - acc: 0.9866 - val_loss: 0.0626 - val_acc: 1.0000
Epoch 2/3
8601/8601 [==============================] - 14s 2ms/sample - loss: 0.0489 - acc: 1.0000 - val_loss: 0.0385 - val_acc: 1.0000
Epoch 3/3
8601/8601 [==============================] - 14s 2ms/sample - loss: 0.0322 - acc: 1.0000 - val_loss: 0.0268 - val_acc: 1.0000
CPU times: user 1min 43s, sys: 42.3 s, total: 2min 25s
Wall time: 44.8 s


## PERFORMANCE ON THE TEST SET


In [99]:
%%time
result = model.evaluate(x_test_pad, rep_test_x)

14448/14448 [==============================] - 12s 835us/sample - loss: 3.6310 - acc: 0.0000e+00
CPU times: user 27.7 s, sys: 13.2 s, total: 40.9 s
Wall time: 12.1 s


In [100]:
print("Accuracy: {0:.2%}".format(result[1]))

Accuracy: 0.00%


In [101]:
text1 = "This movie is fantastic! I really like it because it is so good!"
text2 = "Good movie!"
text3 = "Maybe I like this movie."
text4 = "Meh ..."
text5 = "If I were a drunk teenager then this movie might be good."
text6 = "Bad movie!"
text7 = "Not a good movie!"
text8 = "This movie really sucks! Can I get my money back please?"
texts = [text1, text2, text3, text4, text5, text6, text7, text8]

In [102]:
tokens = tokenizer.texts_to_sequences(texts)

In [103]:
tokens_pad = pad_sequences(tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)
tokens_pad.shape

(8, 56)

In [104]:
model.predict(tokens_pad)

array([[0.9735117 ],
       [0.97351193],
       [0.9735116 ],
       [0.9735123 ],
       [0.97351044],
       [0.9735115 ],
       [0.97351193],
       [0.97350955]], dtype=float32)